In [1]:
%matplotlib inline

In [8]:
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import h5py, pylab, struct
import matplotlib.cm as cm
import conversions as co
import pandas as pd

#### TODO:
* PLOTS

#### Aspirations:
* Wrap the hdf5 reader such that it can read in chunks at a time

#### Notes:
* snap hdf5 structure: f has keys 'Header', 'PartType1,2,3'
* PartTypes have no attrs, keys = "Coordinates", "ParticleIDs", "Velocities" with no attrs
    * Coords and Vels are ndarrays len == 3
    * PT1=DM (20mil x 3), PT2=disk (8mil x 3), PT3=bulge (2.8mil x 3)
* header has attributes with value == 0: 
    * BoxSize, OmegaLambda, Redshift
    * Flag_[Cooling, DoublePrecision, Feedback, IC_Info, Metals, Sfr, StellarAge]
* nonzero attrs: 
    * HubbleParam, Omega0 == 1, Time (in years / 10^9 (????))
    * NumPart_ThisFile == NumPart_Total (array len 6), MassTable (array len 6)

In [149]:
def read_hdf5(path, p_type=1):
    """types: 1=DM, 2=disk, 3=bulge"""
    groups = ["Header", "PartType1", "PartType2", "PartType3"]
    head = {}
    
    f = h5py.File(path, "r")
    h = f["Header"]
    keys = list(h.attrs)
    for key in keys:
        head[key] = h.attrs[key]
    parts = f[groups[p_type]]
    columns = ["x","y","z", "vx", "vy", "vz"]
    df = pd.DataFrame(np.concatenate((parts["Coordinates"], parts["Velocities"]), axis=1),
                     columns=columns, index=parts["ParticleIDs"][:])
    f.close()
    
    return head, df

In [153]:
path = "data/snap_582.hdf5"
h, df = read_hdf5(path, p_type=3)
df

,x,y,z,vx,vy,vz
28639916,-41.364201,-0.868397,-24.016346,94.006210,-4.976634,61.801258
29546391,-40.073154,0.106762,-23.555355,196.836517,-1.480761,-90.483757
29505943,-42.349831,-0.618606,-21.989767,-213.720779,-94.062553,130.978668
30294667,-46.743774,-2.320340,-20.091316,-145.880554,7.030462,65.931801
30159727,-43.725334,-1.985742,-22.726196,-159.082825,22.662899,-105.977768
...,...,...,...,...,...,...
29222458,-13.814735,-14.249855,77.030037,47.659611,-12.432306,-85.064423
29589556,-13.454148,-42.402237,86.330467,-25.662546,-49.453171,35.612667
29151553,-4.812218,-33.069962,104.416122,-104.050850,37.620075,105.977150
29900917,-5.719983,-28.821281,101.478615,-39.821232,-85.115784,3.542554
